In [ ]:
# ======================== CELL 1: FIX DEPENDENCIES ==========================
import subprocess
import sys

print('🔧 Fixing all package dependencies and kernel state...')

# STEP 1: Wipe any incompatible pyarrow
subprocess.run([sys.executable, "-m", "pip", "uninstall", "-y", "pyarrow"], capture_output=True, check=False)

# STEP 2: Install pyarrow and essentials ONLY once before next cell
subprocess.run([sys.executable, "-m", "pip", "install", "-q", "pyarrow==15.0.2", "keybert", "rank-bm25", "evaluate", "faiss-cpu", "protobuf<5.0.0", "sacremoses"], check=True)

# STEP 3: Install bigframes dependencies if required by your notebook/business use
subprocess.run([sys.executable, "-m", "pip", "install", "-q", "rich==13.7.1"], check=True)
subprocess.run([sys.executable, "-m", "pip", "install", "-q", "google-cloud-bigquery-storage>=2.30.0"], check=True)
subprocess.run([sys.executable, "-m", "pip", "install", "-q", "--upgrade", "google-cloud-bigquery>=3.31.0"], check=True)
subprocess.run([sys.executable, "-m", "pip", "install", "-q", "--upgrade", "google-api-core>=2.10.2"], check=True)

print('✅ Dependencies fixed and installed. Now RESTART the kernel and run Cell 2.')


In [1]:
# ===================== CELL 2: SANITY CHECK ALL IMPORTS =====================
import warnings
warnings.filterwarnings("ignore")
print("🔍 Verifying critical imports...")

try:
    from datasets import load_dataset
    print("✅ datasets")
    from sentence_transformers import SentenceTransformer
    print("✅ sentence-transformers")
    from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelForSequenceClassification
    print("✅ transformers")
    import faiss
    print("✅ faiss")
    from keybert import KeyBERT
    print("✅ keybert")
    from rank_bm25 import BM25Okapi
    print("✅ rank-bm25")
    import torch
    print(f"✅ torch (device: {'cuda' if torch.cuda.is_available() else 'cpu'})")
    print("\n🎉 ALL CRITICAL IMPORTS SUCCESSFUL - You can now run your full pipeline!")
except Exception as e:
    print(f"❌ Import failed: {e}\nPlease RESTART THE KERNEL and run Cell 1 again.")
    raise


🔍 Verifying critical imports...
✅ datasets


2025-11-01 05:18:21.039471: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761974301.207939     128 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761974301.253740     128 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

✅ sentence-transformers
✅ transformers
✅ faiss
✅ keybert
✅ rank-bm25
✅ torch (device: cuda)

🎉 ALL CRITICAL IMPORTS SUCCESSFUL - You can now run your full pipeline!


In [2]:
# ============== CELL 3: PRODUCTION-READY MULTI-DOMAIN RAG PIPELINE ==============

import re
import json
import time
import pickle
import random
import logging
from pathlib import Path
from dataclasses import dataclass
from datetime import datetime
from typing import List, Dict, Tuple, Any

import numpy as np
import torch
import faiss
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from sentence_transformers import SentenceTransformer, util
from transformers import (
    AutoTokenizer, 
    AutoModelForCausalLM, 
    AutoModelForSequenceClassification
)
from nltk.tokenize import word_tokenize, sent_tokenize
from rank_bm25 import BM25Okapi
from keybert import KeyBERT

# Setup logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
logger.info(f"🔧 Using device: {device}")

# ============================================================================
# CONFIGURATION - PRODUCTION OPTIMIZED
# ============================================================================

@dataclass
class DomainConfig:
    name: str
    dataset_name: str
    config_name: str = None
    dataset_split: str = "train"
    index_path: str = None
    id2doc_path: str = None
    metadata_path: str = None
    
    def __post_init__(self):
        if self.index_path is None:
            self.index_path = f"{self.name}_faiss.index"
        if self.id2doc_path is None:
            self.id2doc_path = f"{self.name}_id2doc.pkl"
        if self.metadata_path is None:
            self.metadata_path = f"{self.name}_metadata.json"

@dataclass
class RAGConfig:
    embed_model: str = "sentence-transformers/all-MiniLM-L6-v2"
    reranker_model: str = "BAAI/bge-reranker-large"
    hyde_model: str = "microsoft/BioGPT-Large"
    generator_model: str = "microsoft/BioGPT-Large"
    chunk_window: int = 3
    chunk_stride: int = 1
    retrieve_k: int = 30
    rerank_topk: int = 8
    context_chunks: int = 6  # INCREASED from 4
    hyde_weight: float = 0.4
    faiss_alpha: float = 0.6
    max_new_tokens: int = 400  # INCREASED from 200
    hyde_max_tokens: int = 80  # INCREASED from 60
    completeness_threshold: float = 0.65
    faithfulness_threshold: float = 0.55
    retrieval_weight: float = 0.4
    completeness_weight: float = 0.3
    faithfulness_weight: float = 0.3
    prompts_log: str = "prompts_outputs.pkl"
    random_seed: int = 42
    test_size: float = 0.15

DOMAINS = [
    DomainConfig(
        name="women_health",
        dataset_name="altaidevorg/women-health-mini",
        index_path="/kaggle/input/indexes/women_health_faiss.index",
        id2doc_path="/kaggle/input/indexes/women_health_id2doc.pkl",
        metadata_path="/kaggle/input/indexes/women_health_metadata.json"
    ),
    DomainConfig(
        name="medical_qa",
        dataset_name="Malikeh1375/medical-question-answering-datasets",
        config_name="all-processed",
        index_path="/kaggle/input/indexes/medical_qa_faiss.index",
        id2doc_path="/kaggle/input/indexes/medical_qa_id2doc.pkl",
        metadata_path="/kaggle/input/indexes/medical_qa_metadata.json"
    )
]

config = RAGConfig()

# ============================================================================
# UTILITY FUNCTIONS
# ============================================================================

def clean_text_artifacts(text: str) -> str:
    text = re.sub(r"^(Answer:|Final answer:|Response:|Detailed Medical Answer:)\s*", "", text, flags=re.IGNORECASE)
    text = re.sub(r"<\/?[^>]+>|</s>|▃|\[INST\]|\[/INST\]", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text.strip(" \n\r\t\"'")

def monitor_memory():
    if torch.cuda.is_available():
        allocated = torch.cuda.memory_allocated() / 1e9
        total = torch.cuda.get_device_properties(0).total_memory / 1e9
        logger.info(f"💾 GPU: {allocated:.2f}GB / {total:.2f}GB ({allocated/total*100:.1f}%)")
        if allocated/total > 0.85:
            torch.cuda.empty_cache()

# ============================================================================
# DATASET LOADING
# ============================================================================

class DatasetLoader:
    @staticmethod
    def extract_qa_pairs(dataset, domain_name: str) -> list:
        qa_data = []
        for idx, row in enumerate(dataset):
            try:
                if 'conversations' in row and isinstance(row['conversations'], list):
                    conversations = row['conversations']
                    question, answer = "", ""
                    for msg in conversations:
                        if msg.get("role") == "user" and not question:
                            question = msg.get("content", "")
                        if msg.get("role") == "assistant" and not answer:
                            answer = msg.get("content", "")
                    if question and answer:
                        qa_data.append({
                            "question": question.strip(),
                            "answer": answer.strip(),
                            "domain": domain_name,
                            "source_id": idx
                        })
                        continue
                if 'question' in row and 'answer' in row:
                    qa_data.append({
                        "question": str(row['question']).strip(),
                        "answer": str(row['answer']).strip(),
                        "domain": domain_name,
                        "source_id": idx
                    })
                    continue
                if 'input' in row and 'output' in row:
                    qa_data.append({
                        "question": str(row['input']).strip(),
                        "answer": str(row['output']).strip(),
                        "domain": domain_name,
                        "source_id": idx
                    })
            except Exception as e:
                if idx < 2:
                    print(f"extract_qa_pairs WARNING, row {idx}: {e}")
                continue
        return qa_data
    
    @staticmethod
    def load_domain_data(domain_config: DomainConfig) -> tuple:
        logger.info(f"📥 Loading {domain_config.name}...")
        try:
            if domain_config.config_name:
                dataset = load_dataset(domain_config.dataset_name, domain_config.config_name, split=domain_config.dataset_split)
            else:
                dataset = load_dataset(domain_config.dataset_name, split=domain_config.dataset_split)
            logger.info(f"Dataset {domain_config.name} loaded with {len(dataset)} rows")
            qa_data = DatasetLoader.extract_qa_pairs(dataset, domain_config.name)
            if not qa_data:
                logger.error(f"No QA pairs extracted from {domain_config.name}")
                logger.error(f"Sample row structure: {dataset[0]}")
                raise ValueError(f"No QA pairs extracted from {domain_config.name}. Check dataset structure.")
            train_data, test_data = train_test_split(
                qa_data, test_size=config.test_size, random_state=config.random_seed
            )
            logger.info(f"✅ {domain_config.name}: {len(train_data)} train, {len(test_data)} test")
            return train_data, test_data
        except Exception as e:
            logger.error(f"❌ Failed to load {domain_config.name}: {e}")
            raise

# ============================================================================
# TEXT CHUNKING
# ============================================================================

class TextChunker:
    @staticmethod
    def create_chunks(data: List[Dict], window: int = 3, stride: int = 1, min_chars: int = 50) -> List[Dict]:
        chunks = []
        for item in data:
            text = item.get("answer", "")
            if not text or len(text) < min_chars:
                continue
            
            sentences = sent_tokenize(text)
            if not sentences:
                continue
            
            if len(sentences) <= window:
                chunks.append({
                    "chunk": " ".join(sentences),
                    "source_idx": item.get("source_id", -1),
                    "domain": item.get("domain", "unknown"),
                    "chunk_id": len(chunks)
                })
                continue
            
            for i in range(0, max(1, len(sentences) - window + 1), stride):
                chunks.append({
                    "chunk": " ".join(sentences[i:i + window]),
                    "source_idx": item.get("source_id", -1),
                    "domain": item.get("domain", "unknown"),
                    "chunk_id": len(chunks),
                    "window": (i, i + window)
                })
        return chunks

# ============================================================================
# MODEL MANAGEMENT
# ============================================================================

class ModelManager:
    def __init__(self, config: RAGConfig, device: torch.device):
        self.config = config
        self.device = device
        self.models = {}
    
    def load_embedder(self):
        logger.info(f"📦 Loading embedder...")
        embedder = SentenceTransformer(self.config.embed_model, device=self.device)
        self.models['embedder'] = embedder
        logger.info(f"✅ Embedder loaded")
        return embedder
    
    def load_reranker(self):
        logger.info(f"📦 Loading reranker...")
        tokenizer = AutoTokenizer.from_pretrained(self.config.reranker_model)
        model = AutoModelForSequenceClassification.from_pretrained(
            self.config.reranker_model,
            torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32
        ).to(self.device)
        model.eval()
        self.models['reranker_tokenizer'] = tokenizer
        self.models['reranker_model'] = model
        logger.info(f"✅ Reranker loaded")
        return tokenizer, model
    
    def load_hyde_model(self):
        logger.info(f"📦 Loading HyDE model...")
        try:
            tokenizer = AutoTokenizer.from_pretrained(self.config.hyde_model)
            model = AutoModelForCausalLM.from_pretrained(
                self.config.hyde_model,
                torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
                low_cpu_mem_usage=True
            ).to(self.device)
            model.eval()
            if tokenizer.pad_token is None:
                tokenizer.pad_token = tokenizer.eos_token
            self.models['hyde_tokenizer'] = tokenizer
            self.models['hyde_model'] = model
            logger.info(f"✅ HyDE model loaded")
            return tokenizer, model
        except Exception as e:
            logger.warning(f"⚠️ HyDE load failed, using query expansion: {e}")
            return None, None
    
    def load_generator(self):
        logger.info(f"📦 Loading generator...")
        tokenizer = AutoTokenizer.from_pretrained(self.config.generator_model)
        model = AutoModelForCausalLM.from_pretrained(
            self.config.generator_model,
            torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
            low_cpu_mem_usage=True
        ).to(self.device)
        model.eval()
        self.models['gen_tokenizer'] = tokenizer
        self.models['gen_model'] = model
        logger.info(f"✅ Generator loaded")
        return tokenizer, model
    
    def load_keyword_extractor(self):
        try:
            kw_model = KeyBERT(model=self.models.get('embedder'))
            self.models['keyword_extractor'] = kw_model
            logger.info(f"✅ KeyBERT loaded")
            return kw_model
        except Exception as e:
            logger.warning(f"⚠️ KeyBERT load failed: {e}")
            return None
    
    def load_all(self):
        logger.info("🔧 Loading all models...")
        self.load_embedder()
        self.load_reranker()
        self.load_hyde_model()
        self.load_generator()
        self.load_keyword_extractor()
        monitor_memory()
        logger.info("✅ All models loaded")
        return self.models

# ============================================================================
# INDEX MANAGEMENT
# ============================================================================

class MultiDomainIndexManager:
    def __init__(self, config: RAGConfig, embedder: SentenceTransformer):
        self.config = config
        self.embedder = embedder
        self.domain_indices = {}
    
    def build_or_load_domain_index(self, domain_config: DomainConfig, chunks: List[Dict]) -> Tuple[faiss.Index, List[str], BM25Okapi]:
        if Path(domain_config.index_path).exists() and Path(domain_config.id2doc_path).exists():
            try:
                return self._load_existing_index(domain_config)
            except:
                pass
        return self._build_new_index(domain_config, chunks)
    
    def _load_existing_index(self, domain_config: DomainConfig) -> Tuple[faiss.Index, List[str], BM25Okapi]:
        logger.info(f"📂 Loading existing {domain_config.name} index...")
        index = faiss.read_index(domain_config.index_path)
        with open(domain_config.id2doc_path, "rb") as f:
            id2doc = pickle.load(f)
        bm25_corpus = [word_tokenize(doc.lower()) for doc in id2doc]
        bm25 = BM25Okapi(bm25_corpus)
        logger.info(f"✅ Loaded {domain_config.name}: {index.ntotal} vectors")
        return index, id2doc, bm25
    
    def _build_new_index(self, domain_config: DomainConfig, chunks: List[Dict]) -> Tuple[faiss.Index, List[str], BM25Okapi]:
        logger.info(f"🔨 Building {domain_config.name} index...")
        id2doc = [chunk["chunk"] for chunk in chunks]
        
        embeddings = self.embedder.encode(
            id2doc, normalize_embeddings=True, show_progress_bar=True,
            batch_size=64, convert_to_numpy=True
        ).astype('float32')
        
        dim = embeddings.shape[1]
        index = faiss.IndexFlatIP(dim)
        index.add(embeddings)
        
        bm25_corpus = [word_tokenize(doc.lower()) for doc in id2doc]
        bm25 = BM25Okapi(bm25_corpus)
        
        faiss.write_index(index, domain_config.index_path)
        with open(domain_config.id2doc_path, "wb") as f:
            pickle.dump(id2doc, f)
        
        metadata = {"created_at": time.time(), "n_vectors": int(index.ntotal), "embedding_dim": dim, "domain": domain_config.name}
        with open(domain_config.metadata_path, "w") as f:
            json.dump(metadata, f, indent=2)
        
        logger.info(f"✅ Built {domain_config.name}: {index.ntotal} vectors")
        return index, id2doc, bm25
    
    def load_all_domains(self, domain_chunks: Dict[str, List[Dict]]):
        for domain in DOMAINS:
            index, id2doc, bm25 = self.build_or_load_domain_index(domain, domain_chunks.get(domain.name, []))
            self.domain_indices[domain.name] = {
                'index': index, 'id2doc': id2doc, 'bm25': bm25, 'config': domain
            }
        logger.info(f"✅ Loaded {len(self.domain_indices)} domain indices")

# ============================================================================
# QUERY ROUTER
# ============================================================================

class QueryRouter:
    def __init__(self, embedder: SentenceTransformer, domain_indices: Dict):
        self.embedder = embedder
        self.domain_indices = domain_indices
        self.domain_centroids = self._compute_centroids()
    
    def _compute_centroids(self) -> Dict[str, np.ndarray]:
        centroids = {}
        logger.info("🎯 Computing domain centroids...")
        for domain_name, domain_data in self.domain_indices.items():
            id2doc = domain_data['id2doc']
            sample_docs = random.sample(id2doc, min(500, len(id2doc)))
            embeddings = self.embedder.encode(sample_docs, normalize_embeddings=True, convert_to_numpy=True)
            centroids[domain_name] = embeddings.mean(axis=0)
        return centroids
    
    def route_query(self, query: str, top_k: int = 2) -> List[str]:
        query_emb = self.embedder.encode([query], normalize_embeddings=True, convert_to_numpy=True)[0]
        similarities = {domain: float(np.dot(query_emb, centroid)) for domain, centroid in self.domain_centroids.items()}
        sorted_domains = sorted(similarities.items(), key=lambda x: x[1], reverse=True)
        selected = [d[0] for d in sorted_domains[:top_k]]
        logger.info(f"🧭 Routed to: {selected}")
        return selected

# ============================================================================
# MULTI-DOMAIN RAG PIPELINE - PRODUCTION VERSION
# ============================================================================

class MultiDomainRAGPipeline:
    def __init__(self, config: RAGConfig, domains: List[DomainConfig]):
        self.config = config
        self.domains = domains
        self.device = device
        
        self.model_manager = ModelManager(config, device)
        self.models = self.model_manager.load_all()
        
        self.data = {}
        self.test_data = {}
        domain_chunks = {}
        
        for domain in domains:
            train_data, test_data = DatasetLoader.load_domain_data(domain)
            self.data[domain.name] = train_data
            self.test_data[domain.name] = test_data
            chunks = TextChunker.create_chunks(train_data, window=config.chunk_window, stride=config.chunk_stride)
            domain_chunks[domain.name] = chunks
        
        self.index_manager = MultiDomainIndexManager(config, self.models['embedder'])
        self.index_manager.load_all_domains(domain_chunks)
        
        self.router = QueryRouter(self.models['embedder'], self.index_manager.domain_indices)
        self.prompts_log = []
        
        logger.info("✅ Multi-domain RAG pipeline initialized")
    
    def is_medical_query(self, query: str) -> bool:
        """Check if query is medical/health-related. NEW METHOD"""
        medical_keywords = [
            'symptom', 'disease', 'treatment', 'diagnosis', 'health', 'medical',
            'doctor', 'patient', 'pain', 'fever', 'infection', 'condition',
            'medicine', 'drug', 'therapy', 'screening', 'test', 'exam',
            'pregnancy', 'diabetes', 'cancer', 'blood', 'pressure', 'heart',
            'period', 'menstrual', 'hormone', 'pregnant', 'ovary', 'breast'
        ]
        query_lower = query.lower()
        return any(keyword in query_lower for keyword in medical_keywords)
    
    def generate_hyde_answer(self, query: str) -> str:
        if self.models['hyde_model'] is None:
            return query
        
        # IMPROVED PROMPT
        prompt = f"Medical Question: {query}\nDetailed Medical Answer with relevant terminology:"
        try:
            inputs = self.models['hyde_tokenizer'](prompt, return_tensors="pt", truncation=True, max_length=256).to(self.device)
            with torch.no_grad():
                outputs = self.models['hyde_model'].generate(
                    **inputs, max_new_tokens=self.config.hyde_max_tokens,
                    do_sample=False, pad_token_id=self.models['hyde_tokenizer'].eos_token_id,
                    repetition_penalty=1.15
                )
            text = self.models['hyde_tokenizer'].decode(outputs[0], skip_special_tokens=True)
            hyde = clean_text_artifacts(text.split("Answer:")[-1])
            return hyde if hyde else query
        except:
            return query
    
    def retrieve_from_domain(self, query: str, domain_name: str, k: int) -> List[Tuple[int, float, str]]:
        domain_data = self.index_manager.domain_indices[domain_name]
        index = domain_data['index']
        id2doc = domain_data['id2doc']
        bm25 = domain_data['bm25']
        
        hyde_text = self.generate_hyde_answer(query)
        q_emb = self.models['embedder'].encode([query], normalize_embeddings=True, convert_to_numpy=True).astype('float32')
        h_emb = self.models['embedder'].encode([hyde_text], normalize_embeddings=True, convert_to_numpy=True).astype('float32')
        merged_emb = (1 - self.config.hyde_weight) * q_emb + self.config.hyde_weight * h_emb
        
        D, I = index.search(merged_emb, k)
        faiss_scores = D[0]
        if faiss_scores.max() > faiss_scores.min():
            faiss_norm = (faiss_scores - faiss_scores.min()) / (faiss_scores.max() - faiss_scores.min())
        else:
            faiss_norm = np.ones_like(faiss_scores)
        faiss_map = {int(idx): float(score) for idx, score in zip(I[0], faiss_norm)}
        
        bm25_scores = bm25.get_scores(word_tokenize(query.lower()))
        if bm25_scores.max() > bm25_scores.min():
            bm25_norm = (bm25_scores - bm25_scores.min()) / (bm25_scores.max() - bm25_scores.min())
        else:
            bm25_norm = np.zeros_like(bm25_scores)
        
        candidates = set(I[0].tolist()) | set(np.argsort(bm25_scores)[::-1][:k].tolist())
        merged_scores = []
        for idx in candidates:
            f = faiss_map.get(int(idx), 0.0)
            b = float(bm25_norm[int(idx)]) if int(idx) < len(bm25_norm) else 0.0
            score = self.config.faiss_alpha * f + (1 - self.config.faiss_alpha) * b
            merged_scores.append((int(idx), score, domain_name))
        
        merged_scores.sort(key=lambda x: x[1], reverse=True)
        return merged_scores[:k]
    
    def rerank_candidates(self, query: str, candidates: List[Tuple[int, float, str]]) -> List[Tuple[str, float, str]]:
        texts, metadata = [], []
        for idx, score, domain_name in candidates:
            domain_data = self.index_manager.domain_indices[domain_name]
            text = domain_data['id2doc'][idx]
            texts.append(text)
            metadata.append((idx, domain_name))
        
        reranker_scores = []
        batch_size = 8
        for i in range(0, len(texts), batch_size):
            batch_texts = texts[i:i+batch_size]
            inputs = self.models['reranker_tokenizer'](
                [query] * len(batch_texts), batch_texts,
                padding=True, truncation=True, max_length=512, return_tensors="pt"
            ).to(self.device)
            
            with torch.no_grad():
                outputs = self.models['reranker_model'](**inputs)
                logits = outputs.logits.cpu().numpy()
            
            for lg in logits:
                if lg.shape == ():
                    score = float(lg)
                elif len(lg.shape) == 1 and lg.shape[0] == 1:
                    score = float(lg[0])
                elif len(lg.shape) == 1 and lg.shape[0] == 2:
                    score = float(lg[1])
                else:
                    score = float(np.max(lg))
                reranker_scores.append(score)
        
        reranked = [(texts[i], reranker_scores[i], metadata[i][1]) for i in range(len(texts))]
        reranked.sort(key=lambda x: x[1], reverse=True)
        return reranked[:self.config.rerank_topk]
    
    def generate_answer(self, query: str, contexts: List[Tuple[str, float, str]]) -> str:
        context_parts = [f"[Source {i+1} from {domain}]:\n{text}" 
                        for i, (text, score, domain) in enumerate(contexts[:self.config.context_chunks])]
        context_block = "\n\n".join(context_parts)
        
        # IMPROVED PROMPT
        prompt = f"""You are a medical expert assistant. Using the following retrieved medical information, provide a comprehensive, accurate, and helpful answer to the patient's question.

If the question asks about:
- A condition: Identify it clearly and describe symptoms
- Tests: List specific tests and what they measure  
- Treatment: Describe available treatment options with details
- Multiple parts: Address each part of the question systematically

Always base your answer strictly on the provided sources. If information is insufficient, acknowledge that and suggest consulting a healthcare provider.

Medical Context:
{context_block}

Patient Question: {query}

Detailed Medical Answer:"""
        
        try:
            inputs = self.models['gen_tokenizer'](prompt, return_tensors="pt", truncation=True, max_length=1024).to(self.device)
            with torch.no_grad():
                # IMPROVED GENERATION PARAMETERS
                outputs = self.models['gen_model'].generate(
                    **inputs, 
                    max_new_tokens=self.config.max_new_tokens,
                    min_new_tokens=50,
                    do_sample=True,
                    temperature=0.7,
                    top_p=0.9,
                    pad_token_id=self.models['gen_tokenizer'].eos_token_id,
                    repetition_penalty=1.2,
                    no_repeat_ngram_size=3
                )
            raw = self.models['gen_tokenizer'].decode(outputs[0], skip_special_tokens=True)
            answer = clean_text_artifacts(raw.split("Detailed Medical Answer:")[-1])
            
            # POST-PROCESSING
            sentences = sent_tokenize(answer)
            if sentences and len(sentences[-1]) < 20:
                answer = " ".join(sentences[:-1])
            
            if len(answer.split()) < 10:
                answer = "I apologize, but I don't have enough specific information in my knowledge base to provide a complete answer to your question. Please consult with a healthcare professional for personalized medical advice."
            
            self.prompts_log.append({
                "type": "generate", "query": query,
                "contexts": [(t, d) for t, _, d in contexts[:self.config.context_chunks]],
                "prompt": prompt, "raw": raw, "answer": answer, "timestamp": time.time()
            })
            
            return answer if answer else "Insufficient information. Please consult a healthcare professional."
        except Exception as e:
            logger.error(f"Generation failed: {e}")
            return "Error generating answer. Please consult a healthcare professional."
    
    def compute_metrics(self, query: str, answer: str, contexts: List[Tuple[str, float, str]]) -> Dict[str, float]:
        metrics = {}
        
        # FIXED RETRIEVAL NORMALIZATION
        if contexts:
            raw_scores = [score for _, score, _ in contexts[:self.config.context_chunks]]
            retrieval_score = np.mean(raw_scores)
            metrics['retrieval'] = float(np.clip((retrieval_score + 10) / 20, 0, 1))
        else:
            metrics['retrieval'] = 0.0
        
        try:
            context_texts = [text for text, _, _ in contexts[:self.config.context_chunks]]
            all_keywords = []
            
            if self.models['keyword_extractor']:
                for ctx_text in context_texts:
                    keywords = self.models['keyword_extractor'].extract_keywords(
                        ctx_text, keyphrase_ngram_range=(1, 2), stop_words='english', top_n=5
                    )
                    all_keywords.extend([kw for kw, _ in keywords])
            
            unique_keywords = list(dict.fromkeys([kw.lower() for kw in all_keywords if kw]))
            
            if unique_keywords and answer:
                answer_emb = self.models['embedder'].encode([answer], normalize_embeddings=True, convert_to_tensor=True)
                keyword_embs = self.models['embedder'].encode(unique_keywords, normalize_embeddings=True, convert_to_tensor=True)
                similarities = util.cos_sim(answer_emb, keyword_embs).cpu().numpy()[0]
                covered = (similarities >= self.config.completeness_threshold).sum()
                metrics['completeness'] = float(covered / len(unique_keywords))
            else:
                metrics['completeness'] = 0.0
        except:
            metrics['completeness'] = 0.0
        
        try:
            if answer and contexts:
                answer_sentences = sent_tokenize(answer)
                context_sentences = []
                for text, _, _ in contexts[:self.config.context_chunks]:
                    context_sentences.extend(sent_tokenize(text))
                
                if answer_sentences and context_sentences:
                    ans_embs = self.models['embedder'].encode(answer_sentences, normalize_embeddings=True, convert_to_tensor=True)
                    ctx_embs = self.models['embedder'].encode(context_sentences, normalize_embeddings=True, convert_to_tensor=True)
                    sim_matrix = util.cos_sim(ans_embs, ctx_embs).cpu().numpy()
                    max_sims = np.max(sim_matrix, axis=1)
                    faithful = (max_sims >= self.config.faithfulness_threshold).sum()
                    metrics['faithfulness'] = float(faithful / len(answer_sentences))
                else:
                    metrics['faithfulness'] = 0.0
            else:
                metrics['faithfulness'] = 0.0
        except:
            metrics['faithfulness'] = 0.0
        
        metrics['composite'] = (
            self.config.retrieval_weight * metrics['retrieval'] +
            self.config.completeness_weight * metrics['completeness'] +
            self.config.faithfulness_weight * metrics['faithfulness']
        )
        
        return metrics
    
    def run_query(self, query: str, top_domains: int = 2, log_diagnostics: bool = False) -> Dict[str, Any]:
        logger.info(f"🔍 Processing: {query[:100]}...")
        
        # OUT-OF-DOMAIN CHECK
        if not self.is_medical_query(query):
            return {
                "query": query,
                "routed_domains": [],
                "answer": "I apologize, but I'm designed to answer medical and health-related questions. Your question doesn't appear to be medical in nature. Could you please ask a health or medical question?",
                "contexts": [],
                "metrics": {"retrieval": 0.0, "completeness": 0.0, "faithfulness": 0.0, "composite": 0.0}
            }
        
        selected_domains = self.router.route_query(query, top_k=top_domains)
        
        all_candidates = []
        for domain_name in selected_domains:
            candidates = self.retrieve_from_domain(query, domain_name, k=self.config.retrieve_k)
            all_candidates.extend(candidates)
        
        if log_diagnostics:
            logger.info(f"Retrieved {len(all_candidates)} candidates from {len(selected_domains)} domains")
        
        reranked = self.rerank_candidates(query, all_candidates)
        
        if log_diagnostics:
            logger.info("Top reranked contexts:")
            for i, (text, score, domain) in enumerate(reranked[:3]):
                logger.info(f"  {i+1}. [{domain}] (score={score:.3f}): {text[:150]}...")
        
        answer = self.generate_answer(query, reranked)
        metrics = self.compute_metrics(query, answer, reranked)
        
        result = {
            "query": query,
            "routed_domains": selected_domains,
            "answer": answer,
            "contexts": [(text, domain) for text, _, domain in reranked[:self.config.context_chunks]],
            "metrics": metrics
        }
        
        return result
    
    def evaluate_batch(self, queries: List[str], log_diagnostics: bool = False) -> Dict[str, Any]:
        logger.info(f"📊 Evaluating {len(queries)} queries...")
        
        results = []
        failed = []
        
        for i, query in enumerate(queries):
            try:
                result = self.run_query(query, log_diagnostics=log_diagnostics)
                results.append(result)
                
                if (i + 1) % 3 == 0:
                    logger.info(f"Progress: {i+1}/{len(queries)}")
                    monitor_memory()
            except Exception as e:
                logger.error(f"Failed query {i}: {e}")
                failed.append((i, query, str(e)))
        
        if not results:
            return {"error": "No successful queries"}
        
        avg_metrics = {
            "retrieval": np.mean([r["metrics"]["retrieval"] for r in results]),
            "completeness": np.mean([r["metrics"]["completeness"] for r in results]),
            "faithfulness": np.mean([r["metrics"]["faithfulness"] for r in results]),
            "composite": np.mean([r["metrics"]["composite"] for r in results])
        }
        
        summary = {
            "total_queries": len(queries),
            "successful": len(results),
            "failed": len(failed),
            "success_rate": len(results) / len(queries),
            "average_metrics": avg_metrics,
            "failed_queries": failed,
            "individual_results": results
        }
        
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        results_file = f"evaluation_{timestamp}.json"
        try:
            with open(results_file, "w") as f:
                json.dump(summary, f, indent=2, default=str)
            logger.info(f"💾 Results saved to {results_file}")
        except:
            pass
        
        return summary

# ============================================================================
# HELPER FUNCTIONS FOR EXECUTION
# ============================================================================

def load_queries_from_file(file_path: str) -> List[str]:
    """Load queries from a text file (one query per line)."""
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            queries = [line.strip() for line in f if line.strip()]
        logger.info(f"✅ Loaded {len(queries)} queries from {file_path}")
        return queries
    except Exception as e:
        logger.error(f"❌ Failed to load queries from {file_path}: {e}")
        return []

def run_interactive_mode(pipeline):
    """Interactive mode: user enters queries one by one."""
    print("\n" + "="*80)
    print("🎯 INTERACTIVE QUERY MODE")
    print("="*80)
    print("Enter your medical queries (type 'quit' or 'exit' to stop):\n")
    
    while True:
        query = input("Query: ").strip()
        if query.lower() in ['quit', 'exit', 'q']:
            print("👋 Exiting interactive mode.")
            break
        if not query:
            continue
            
        result = pipeline.run_query(query, top_domains=2, log_diagnostics=False)
        print("\n" + "-"*80)
        print(f"QUERY: {result['query']}")
        print(f"Routed to: {result['routed_domains']}")
        print(f"Answer: {result['answer']}")
        print("\n📊 METRICS:")
        for metric_name, value in result['metrics'].items():
            print(f"  {metric_name}: {value:.3f}")
        print("-"*80 + "\n")

def run_batch_from_list(pipeline, queries: List[str], show_individual: bool = True):
    """Run batch evaluation on a list of queries."""
    print("\n" + "="*80)
    print(f"📊 BATCH EVALUATION MODE ({len(queries)} queries)")
    print("="*80 + "\n")
    
    if show_individual:
        for i, query in enumerate(queries, 1):
            print(f"\n[Query {i}/{len(queries)}]")
            result = pipeline.run_query(query, top_domains=2, log_diagnostics=False)
            print(f"Q: {result['query']}")
            print(f"Routed to: {result['routed_domains']}")
            print(f"A: {result['answer']}")
            print("Metrics:", end=" ")
            for metric_name, value in result['metrics'].items():
                print(f"{metric_name}={value:.3f}", end=" ")
            print("\n" + "-"*80)
    
    batch_results = pipeline.evaluate_batch(queries, log_diagnostics=False)
    print("\n📈 BATCH EVALUATION SUMMARY")
    print("="*80)
    print(f"Success Rate: {batch_results['success_rate']:.1%}")
    print(f"Average Retrieval: {batch_results['average_metrics']['retrieval']:.3f}")
    print(f"Average Completeness: {batch_results['average_metrics']['completeness']:.3f}")
    print(f"Average Faithfulness: {batch_results['average_metrics']['faithfulness']:.3f}")
    print(f"Average Composite: {batch_results['average_metrics']['composite']:.3f}")
    print("="*80)
    
    return batch_results

print("="*80)
print("🚀 INITIALIZING MULTI-DOMAIN RAG PIPELINE")
print("="*80)

rag_pipeline = MultiDomainRAGPipeline(config, DOMAINS)

print("\n✅ Pipeline ready! Run the next cell to execute queries.")


🚀 INITIALIZING MULTI-DOMAIN RAG PIPELINE


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

all-processed/train-00000-of-00001-9bfe4(…):   0%|          | 0.00/160M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/246678 [00:00<?, ? examples/s]

Batches:   0%|          | 0/16 [00:00<?, ?it/s]

Batches:   0%|          | 0/16 [00:00<?, ?it/s]


✅ Pipeline ready! Run the next cell to execute queries.


In [ ]:
# ============================================================================
# CELL 4: EXECUTION MODES - CHANGE FLAGS AND RERUN THIS CELL
# ============================================================================

# Mode 1: Interactive (user input)
RUN_INTERACTIVE = True  

# Mode 2: Batch from file
RUN_FROM_FILE = False
QUERIES_FILE = "queries.txt"

# Mode 3: Batch from predefined list
RUN_DEMO_BATCH = False
DEMO_QUERIES = [
    "What are the recommended health screenings for women in their 40s?",
    "I'm 35 years old with irregular periods and excessive facial hair. What condition might this indicate, what tests should I get, and what are the treatment options?",
    "How does gestational diabetes differ from Type 2 diabetes, and what are the specific screening guidelines for pregnant women?"
]

# Mode 4: Single query demo
RUN_SINGLE_DEMO = False
SINGLE_QUERY = "What are the symptoms of menopause?"

# ========================================================================
# EXECUTE
# ========================================================================

if RUN_INTERACTIVE:
    run_interactive_mode(rag_pipeline)

elif RUN_FROM_FILE:
    queries = load_queries_from_file(QUERIES_FILE)
    if queries:
        batch_results = run_batch_from_list(rag_pipeline, queries, show_individual=True)

elif RUN_DEMO_BATCH:
    batch_results = run_batch_from_list(rag_pipeline, DEMO_QUERIES, show_individual=True)

elif RUN_SINGLE_DEMO:
    result = rag_pipeline.run_query(SINGLE_QUERY, top_domains=2, log_diagnostics=True)
    print("\n==================")
    print(f"QUERY: {result['query']}")
    print("==================")
    print(f"Routed to: {result['routed_domains']}")
    print(f"Answer: {result['answer']}")
    print("\n📊 METRICS:")
    for metric_name, value in result['metrics'].items():
        print(f"  {metric_name}: {value:.3f}")

else:
    print("⚠️ No execution mode selected. Set one of the RUN_* flags to True.")

try:
    with open(config.prompts_log, "wb") as f:
        pickle.dump(rag_pipeline.prompts_log, f)
    print(f"\n📝 Prompt logs saved to {config.prompts_log}")
except Exception as e:
    logger.warning(f"Could not save prompt logs: {e}")

print("\n✅ EXECUTION COMPLETE")
monitor_memory()



🎯 INTERACTIVE QUERY MODE
Enter your medical queries (type 'quit' or 'exit' to stop):



Query:  what is neurology



--------------------------------------------------------------------------------
QUERY: what is neurology
Routed to: []
Answer: I apologize, but I'm designed to answer medical and health-related questions. Your question doesn't appear to be medical in nature. Could you please ask a health or medical question?

📊 METRICS:
  retrieval: 0.000
  completeness: 0.000
  faithfulness: 0.000
  composite: 0.000
--------------------------------------------------------------------------------



Query:  im ill,what medication should i tale



--------------------------------------------------------------------------------
QUERY: im ill,what medication should i tale
Routed to: []
Answer: I apologize, but I'm designed to answer medical and health-related questions. Your question doesn't appear to be medical in nature. Could you please ask a health or medical question?

📊 METRICS:
  retrieval: 0.000
  completeness: 0.000
  faithfulness: 0.000
  composite: 0.000
--------------------------------------------------------------------------------

